In [ ]:
import requests
import json
import datetime
import random

import pandas as pd
import geopandas as gpd

import folium

from concurrent.futures import ThreadPoolExecutor, as_completed

import warnings
warnings.filterwarnings('ignore')

## CreateSession

In [ ]:
url = 'https://pflow-api.csis.u-tokyo.ac.jp/webapi/CreateSession'

data = {
    'UserID': 'JoRAS_Account_ID',
    'Password': 'JoRAS_Account_PWD'
}

response = requests.post(url, data=data, verify=False)

if response.status_code == 200:
    print('Session created successfully')
    print(response.text)
    status, sessionid = response.text.split(',')
    headers = {'WebApiSessionID': sessionid.strip('\r\n')}
    print("Status:{}, SessionID: {}".format(status, sessionid))
else:
    print('Failed to post data:', response.status_code)

## GetNearestRoadPoint

In [ ]:
starttime = datetime.datetime.now()

url = 'http://157.82.223.35/webapi/GetNearestRoadPoint'

data = {
    'UnitTypeCode': '2',
    'PosLongitude': '139.8732',
    'PosLatitude': '36.0484',
    'RoadKindCode': '',
    'RoadNo': '',
}

response = requests.post(url, cookies=headers, data=data)

if response.status_code == 200:
    status, roadtype, roadid, meshcode, lon, lat = [value for s in response.text.split(',') for value in s.split('\r\n') if value]
    if status.strip('\r\n') == '1':
        print("NearestRoadPoint Detected")
        print("Status:{}, RoadType: {}, RoadID: {}, MeshCode: {}, Longitude: {}, Latitude: {}".format(status, roadtype, roadid, meshcode, lon, lat))
else:
    print('Failed to post data:', response.status_code)

endtime = datetime.datetime.now()
print("Get Nearest Road Point time:", endtime-starttime)

## GetRoadRoute

In [ ]:
starttime = datetime.datetime.now()

url = 'http://157.82.223.35/webapi/GetRoadRoute'

data = {
    'UnitTypeCode': '2',
    'StartLongitude': '139.67727019',
    'StartLatitude': '35.66412606',
    'GoalLongitude': '138.808493',
    'GoalLatitude': '36.017459',
    'WayLongitude': '',
    'WayLatitude': '',
    'RoadKindCode': '',
    'RoadNo': '',
    'TransportCode': '6',
    'OutputNum': '1',
}

response = requests.post(url, cookies=headers, data=data, verify=False)

if response.status_code == 200:
    # print(response.text)
    # print([value.strip() for s in response.text.split(',') for value in s.split('\n') if value.strip()])
    split_values = [value.strip() for s in response.text.split(',') for value in s.split('\n') if value.strip()]
    route_values = split_values[11:]

    routes = []
    current_route = None
    current_coordinates = []

    for value in route_values:
        if value.isdigit():
            if current_route is not None:
                routes.append((current_route, current_coordinates))
            current_route = value
            current_coordinates = []
        else:
            current_coordinates.append(value)
    if current_route is not None:
        routes.append((current_route, current_coordinates))

    # print('route:', routes)
    # status, routeno, route1_len,route2_len,route3_len, route1_no, route2_no, route3_no, route1_time, route2_time, route3_time, out_route_no, lon, lat = [value.strip() for s in response.text.split(',') for value in s.split('\n') if value.strip()]

    if status.strip('\r\n') == '1':
        print("NearestRoadPoint Detected")
        # print("Status:{}, RoadType: {}, RoadID: {}, MeshCode: {}, Longitude: {}, Latitude: {}".format(status, roadtype, roadid, meshcode, lon, lat))
else:
    print('Failed to post data:', response.status_code)

endtime = datetime.datetime.now()
print("Get Nearest Road Point time:", endtime-starttime)